<h1><font=18 align=center>Toronto Neighborhood Analysis!</font></h1>
<h2><font=15 align=center>Part 2 - Adding GeoData</h2>
This project will analyze Toronto neighborhood data using data from the below Wikipedia page.  This notebook continues the project by importing geocoder data and adding to to data frame created in part 1.


In [69]:
##import necessary modules
import pandas as pd
from bs4 import BeautifulSoup
import requests
#import geocoder # import geocoder - couldn't get it to work
import json

In [3]:
#scrape wikipedia table
url=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup=url.text
#parse wiki data
wiki_html=BeautifulSoup(soup,'html.parser')

#instantiate data frame
columns=["Postal Code", "Borough", "Neighborhood"]
df_toronto=pd.DataFrame(columns=columns) 

#loop through table rows, slice table header
for tr in wiki_html.table.find_all('tr')[1:]:
        #set vars
        pc=tr.find_all('td')[0].get_text().rstrip()
        borough=tr.find_all('td')[1].get_text().rstrip()
        nh=tr.find_all('td')[2].get_text().rstrip()
        
        #Verify data.  Boroughs not assigned should be ignored.  Postal codes with multiple
        #neighbohoods should be combined
        if "Not assigned" not in borough:
            if pc not in df_toronto['Postal Code'].values:
                df_row={'Postal Code' : pc,'Borough' : borough,'Neighborhood' : nh}
                df_toronto=df_toronto.append(df_row, ignore_index=True)
            elif pc in df_toronto['Postal Code'].values:
                dfloc=df_toronto.loc[df_toronto['Postal Code'] == pc].index
                df_toronto.loc[dfloc,'Neighborhood']=df_toronto.loc[dfloc,'Neighborhood'] + ',' + nh
                #print('PC:', pc, 'Neigborhood:', nh)

In [4]:
#instantiate geo data frame and add lat/long columns
import numpy as np
df_toronto_geo=df_toronto.copy(deep=True)
df_toronto_geo['Latitude']=np.nan
df_toronto_geo['Longitude']=np.nan

In [6]:
df_toronto_geo.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,0.0,0.0
1,M4A,North York,Victoria Village,0.0,0.0
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",0.0,0.0
3,M6A,North York,"Lawrence Manor, Lawrence Heights",0.0,0.0
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",0.0,0.0


<h2><font=10 align=center>Import geoData from geocoder module</font></h2>

In [102]:
#import file handler and define getgeodata function for pulling geodata from google api
import os
#define api key
f=open('gk','r')
key=f.readline()
f.close

def getgeodata(postal_code,key):
    #generate url
    apiurl='https://maps.googleapis.com/maps/api/geocode/json?address={},+Toronto,+Canada&key={}'.format(postal_code,key)   
 
    #create a file to write
    fname='{}.json'.format(postal_code)
    if os.path.exists(fname):
        f=open(fname,'r')
        jsondata=f.read()
        f.close
        return jsondata
    else:
        f=open(fname,'x')
        url=requests.get(apiurl)
        jsondata=url.text
        f.write(jsondata)
        f.close
        return jsondata

In [105]:
#loop through dataframe, get geoData from google api and update dataframe.
for pc in df_toronto_geo['Postal Code']:
    dfloc=df_toronto_geo.loc[df_toronto_geo['Postal Code'] == pc].index #set dataframe location
    json_data=json.loads(getgeodata(pc,key)) #load json_data
    latitude=json_data['results'][0]['geometry']['location']['lat']
    longitude=json_data['results'][0]['geometry']['location']['lng']
    #write latitude/longitude
    df_toronto_geo.loc[dfloc,'Latitude']=latitude
    df_toronto_geo.loc[dfloc,'Longitude']=longitude
    #df_toronto_geo.loc[df_toronto_geo['Postal Code'] == pc]

In [111]:
df_toronto_geo.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [109]:
df_toronto_geo.shape

(103, 5)